# Modelling most important features

## Model Selection

In [1]:
# Importing the libraries
import os
import pandas as pd

## Data Preprocessing

In [2]:
# Load and preprocess the train data 
from pipelines import preprocessing_pipeline

data_file = os.path.join('..', '..', '..', '..', 'data', 'raw', 'train.csv')
train_data = pd.read_csv(data_file, index_col=0, low_memory=False)
train_data = preprocessing_pipeline.fit_transform(train_data)
train_data.head()

,day_phase,bg-5:45,bg-5:30,bg-5:15,bg-5:00,bg-4:45,bg-4:30,bg-4:15,bg-4:00,bg-3:45,...,bg-2:00,bg-1:45,bg-1:30,bg-1:15,bg-1:00,bg-0:45,bg-0:30,bg-0:15,bg-0:00,bg+1:00
id,,,,,,,,,,,,,,,,,,,,,
p01_0,morning,9.6,9.7,9.2,8.7,8.4,8.1,8.3,9.6,11.1,...,17.2,18.2,18.4,18.0,17.3,17.5,17.3,16.2,15.1,13.4
p01_1,morning,9.7,9.2,8.7,8.4,8.1,8.3,9.6,11.1,11.8,...,18.2,18.4,18.0,17.3,17.5,17.3,16.2,15.1,14.4,12.8
p01_2,morning,9.2,8.7,8.4,8.1,8.3,9.6,11.1,11.8,12.8,...,18.4,18.0,17.3,17.5,17.3,16.2,15.1,14.4,13.9,15.5
p01_3,morning,8.7,8.4,8.1,8.3,9.6,11.1,11.8,12.8,13.9,...,18.0,17.3,17.5,17.3,16.2,15.1,14.4,13.9,13.8,14.8
p01_4,morning,8.4,8.1,8.3,9.6,11.1,11.8,12.8,13.9,14.2,...,17.3,17.5,17.3,16.2,15.1,14.4,13.9,13.8,13.4,12.7


## Data Splitting

In [3]:
from sklearn.model_selection import train_test_split

X = train_data.drop(columns=['bg+1:00'])
y = train_data['bg+1:00']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Standardization

In [4]:
from pipelines import standardization_pipeline

X_train = standardization_pipeline.fit_transform(X_train)
X_test = standardization_pipeline.transform(X_test)

X_train.head()

,bg-5:45,bg-5:30,bg-5:15,bg-5:00,bg-4:45,bg-4:30,bg-4:15,bg-4:00,bg-3:45,bg-3:30,...,bg-1:00,bg-0:45,bg-0:30,bg-0:15,bg-0:00,day_phase_evening,day_phase_late_evening,day_phase_morning,day_phase_night,day_phase_noon
id,,,,,,,,,,,,,,,,,,,,,
p12_24040,-0.493555,-0.493993,-0.527091,-0.693587,-0.859729,-1.026852,-0.993513,-1.127463,-1.093195,-0.758764,...,-0.593126,-0.659985,-0.759862,-0.826287,-0.826509,0,0,0,0,1
p12_13267,-0.028365,-0.061718,-0.127602,-0.427031,-0.326309,-0.359835,-0.426388,-0.292770,-0.192118,-0.091541,...,-0.159478,-0.259754,-0.259428,-0.359421,-0.459549,0,0,1,0,0
p03_131,0.769103,0.869334,1.037574,1.238944,1.473981,1.474461,1.208266,0.875799,1.042692,1.242906,...,2.175554,2.108275,2.176018,2.241694,1.875652,0,0,0,0,0
p12_20861,0.569736,0.935838,0.871120,0.672513,0.307126,0.607339,0.908024,1.042737,1.142811,1.342989,...,-0.426338,-0.693337,-0.593050,-0.559506,-0.993309,0,0,0,0,0
p12_24627,-0.991972,-1.059275,-0.859999,-0.726907,-0.593019,-0.459887,-0.126145,0.174657,-0.158745,-0.758764,...,-0.526411,-0.493222,-0.259428,-0.326073,-0.426189,0,0,0,0,0


## Model Selection with LazyPredict

In [5]:
from notebooks.helpers.LazyPredict import get_lazy_regressor

reg = get_lazy_regressor(exclude=['SVR'])
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

 97%|█████████▋| 37/38 [08:58<00:08,  8.49s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003469 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6090
[LightGBM] [Info] Number of data points in the train set: 141619, number of used features: 29
[LightGBM] [Info] Start training from score 8.276012


100%|██████████| 38/38 [08:59<00:00, 14.21s/it]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
KNeighborsRegressor,0.82,0.82,1.29,4.05
ExtraTreesRegressor,0.72,0.72,1.59,65.94
BaggingRegressor,0.63,0.63,1.83,19.32
XGBRegressor,0.59,0.59,1.94,0.59
MLPRegressor,0.56,0.56,2.00,29.52
LGBMRegressor,0.55,0.55,2.01,0.94
HistGradientBoostingRegressor,0.55,0.55,2.01,1.56
GradientBoostingRegressor,0.53,0.53,2.07,33.80
LassoLarsCV,0.50,0.50,2.13,0.44


### We select the best models with RMSE < 2.0

In [7]:
models[models['RMSE'] < 2.0]

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
KNeighborsRegressor,0.82,0.82,1.29,4.05
ExtraTreesRegressor,0.72,0.72,1.59,65.94
BaggingRegressor,0.63,0.63,1.83,19.32
XGBRegressor,0.59,0.59,1.94,0.59
